In [ ]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

CUDA available: True
Number of GPUs available: 2
GPU 0: NVIDIA GeForce RTX 2070
GPU 1: NVIDIA GeForce RTX 2070
GPU 0 memory allocated: 0.00 GB
GPU 0 memory reserved: 0.00 GB
GPU 1 memory allocated: 0.00 GB
GPU 1 memory reserved: 0.00 GB


In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType, NativeDataType

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, CellProperties, DilationedCytoMask ,\
                                    DisplaySequentialParams, IlluminationCorrection, Calculate_BIGFISH_Threshold, rescale_images,\
                                    Automatic_BIGFISH_Threshold

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [ ]:
scope = ScopeClass() 
data = DataContainer(clear_after_error=False)
settings = Settings(name='demo')
experiment = Experiment()

In [ ]:
# you can change the parameters
experiment.initial_data_location = r'Users/Jack/HistoneModification_ScratchAssay/Full 24 hour scratch assay.tif'

experiment.nucChannel = 0
experiment.cytoChannel = 0
experiment.voxel_size_z = 500

settings.num_chunks_to_run = 5 # this is the limit to how many chunks you want to run and it will stop after that
settings.sigma_dict = {0: 50, 1: 100, 2: 100}

Parameters().get_parameters()

{'voxel_size_yx': 100,
 'spot_z': 360,
 'spot_yx': 130,
 'microscope_saving_format': 'pycromanager',
 'state': 'global',
 'local_dataset_location': ['C:\\Users\\formanj\\GitHub\\AngelFISH\\dataBases\\DUSP1_DexTimeConcSweep_10nM_75min_041223'],
 'h5_file': None,
 'total_num_chunks': None,
 'images': None,
 'masks': None,
 'temp': <TemporaryDirectory 'c:\\Users\\formanj\\GitHub\\AngelFISH\\dev\\tmp02lutm0n'>,
 'clear_after_error': False,
 'name': 'demo',
 'return_data_to_NAS': True,
 'NUMBER_OF_CORES': 4,
 'save_files': True,
 'num_chunks_to_run': 5,
 'download_data_from_NAS': True,
 'connection_config_location': 'c:\\Users\\formanj\\GitHub\\AngelFISH\\config_nas.yml',
 'share_name': 'share',
 'display_plots': True,
 'load_in_mask': True,
 'mask_structure': {'masks': ('ptczyx', None, None),
  'cell_mask': ('zyx', 'cytoChannel', 'masks'),
  'nuc_mask': ('zyx', 'nucChannel', 'masks')},
 'order': 'pt',
 'sigma_dict': {0: 50, 1: 100, 2: 100},
 'initial_data_location': ['smFISH_images/Eric_sm

In [ ]:
SingleTiff2NativeDataType().run()

In [ ]:
Display().displayImage_maxProject(channel=2)

In [ ]:
settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90]
settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

SimpleCellposeSegmentaion().run()

In [ ]:
CellProperties().run()